#Version 3.6.0
<hr>
Report of forward time spent on MNIST and CIFAR10. Five models for each dataset.

1. Base Model
2. TRL only Model
3. TCL+TRL Model
4. ChunkReshape map type 1 + TCL + TRL Model
5. ChunkReshape map type 2 + TCL + TRL Model

<hr>

## Issues:

- Study the case of inconsistency in Forward time reports

- Reshape time still remains longer

## Attempts :

- Try to generate and empty output from init

- Try to replicate the process using built-in functions

The result of these attempts are reported at the last code blocks

#Reshape Tensor to a higher dimensional Tensor

In this notetbook, we study the case of reshaping a tensor (i.e 3-Tensor) to a 
higher dimensional tensor (i.e 6-Tensor) and its effect on Tensor Contraction and Regression Layer as previously studied at official Tensorly documentation website.


The process is as follow:
1. Given a n-dimensional tensor, there are n modes and the first mode (mode-1) is considered to be the mode responsible for handling the batch index.
2. There are multiple fibers and the goal is to break them into different chunks and rearrange them in a higher dimensional tensor.
3. Breaking the fibers with coefficients such as $l_1$, etc; Whereas each mode $i$ is split into $l_i$ equal length chunks.
4. The newly made tensor is of the double dimension of the original tensor, for example if the original tensor is 3-dimensional then the new tensor is 6-dimensional.
5. We study to approaches in spliting the modes:
  - normal split  :  $[0,\dots,l_i],[l_i+1,\dots,2l_i],\dots$
  - down sampling :  Usually for the cases when $l_i$ is 2, odd and even indices. 
6. According to the types of splits there are 2 mapping functions (3-Tensor):
  - map 1 : $B(i_1,i_2,i_3,j_1,j_2,j_3) = A(j_1 + (i_1 - 1)M_1, j_2 + (i_2 - 1)M_2, j_3 + (i_3 - 1)M_3)$
  - map 2 : $B(i_1,i_2,i_3,j_1,j_2,j_3) = A(M_1(j_1 - 1) + i_1, M_2(j_2 - 1) + i_2), M_3(j_3 - 1) + i_3$
  - Where $M_i$ is $\frac{I_i}{l_i}$, and $I_i$ is the original size of model $i$.
7. We can easily expand the maps to fit n-dimensional tensors as well.


For this case we create ChunkReshape class where we can do the just that. 

In [133]:
# Necessary Packages 
!pip install tensorly-torch 
!pip install tensorly

import tltorch # TCL and TRL
import tensorly as tl # Tensor operation 
import torch   #  Neural Network
from torch import nn # Neural Network
from torch.autograd import Variable # Tensor input
import torch.optim as optim # Optimization
from torchvision import datasets, transforms # Datasets and transoforms
import torchvision # Data Loader
import torch.nn.functional as F # Activation Functions

import numpy as np # Numerical operations
import itertools # Generate indices

import matplotlib.pyplot as plt # Generate plots
import pandas as pd # Save the result as a sheet

import time # execuation time
import os # to save the results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Original Chunk Reshape

Original as in the first model to be created.

In [3]:
class chunkReshape(nn.Module):
  """Tensor higher dimensional reshape- smaller chunk reshape

    Parameters
    ----------
    input_size : int iterable
        shape of the input, excluding batch size
    L : int list or int
        split coefficients for each mode, excluding batch size
    
    References
    ----------
     N/A
    """

  def __init__(self, input_size, L, map_type = 1, device=None, dtype=None):
      # Input contains the batch size mode
      # L doesnt contain the batch size mode

      super().__init__()

      if isinstance(input_size, int):
        self.input_size = (input_size, )
      else:
        self.input_size = tuple(input_size)
      if isinstance(L, int):
        self.L = (L, )
      else:
        self.L = tuple(L)
      if map_type == 1 or self.L.count(2) != len(self.L):
          self.map_type = 1
      else:
          self.map_type = 2

      self.device = device
      self.dtype  = dtype


      # many operations in init

      self.M = []
      for i,_ in enumerate(self.L):
          if self.input_size[i] % self.L[i] != 0:
              raise RuntimeError(f'Bad L : input_size {self.input_size[i]} is not divisible by L size {self.L[i]}')
          self.M.append(int(self.input_size[i] / self.L[i]))

      self.lists_indices = []
      for i in range(len((list(self.L) + self.M))):
          self.lists_indices.append(np.arange((list(self.L) + self.M)[i]))


      self.indices = np.array([p for p in itertools.product(*self.lists_indices)])

  def forward(self, x):
    """Performs a forward pass"""
    
    # generate output shape
    new_shapes = tuple([x.shape[0]] + list(self.L) + self.M)
    out = torch.empty(new_shapes, device=self.device, dtype=self.dtype)

    # generate all possible combination of indices

    # Already generated in init

    # Mapping

    if self.map_type == 1:
      
      for index_B in self.indices:

          # Input is : x
          # Output is : out
          # Maps via exec() command
          ### Command to execute
          code = '' + 'out[:,'
          map_B_string = ','.join(map(str, index_B))
          code += map_B_string + ']'
          ### Command to execute

          index_A = []
          for ii in range(len(index_B)):
              index_A.append(index_B[ii]*self.M[ii] +  index_B[ii+len(self.L)])
              if ii == len(self.L) - 1:
                  ### Command to execute
                  map_A_string = ','.join(map(str, index_A))
                  code += '= x[:,'
                  code += map_A_string + ']'
                  ### Command to execute
                  # If the original dim is 3D then the 
                  # code is similar to : out[:,0,0,0,0,0,0] = x[:,0,0,0]
                  exec(code)
                  break  

      return out
      
    else:

      for index_B in self.indices:

          # Input is : x
          # Output is : out
          # Maps via exec() command
          ### Command to execute
          code = '' + 'out[:,'
          map_B_string = ','.join(map(str, index_B))
          code += map_B_string + ']'
          ### Command to execute

          index_A = []
          for ii in range(len(index_B)):
              index_A.append(index_B[ii + len(self.L)]*2 +  index_B[ii])
              if ii == len(self.L) - 1:
                  ### Command to execute
                  map_A_string = ','.join(map(str, index_A))
                  code += '= x[:,'
                  code += map_A_string + ']'
                  ### Command to execute
                  # If the original dim is 3D then the 
                  # code is similar to : out[:,0,0,0,0,0,0] = x[:,0,0,0]
                  exec(code)
                  break  
      return out


# Version  2 of chunk reshape

Another version of chunk reshape to study the case of creating output from the init and then calculating the time spent.

In [22]:
class chunkReshape_version2(nn.Module):
  """Tensor higher dimensional reshape- smaller chunk reshape

    Parameters
    ----------
    input_size : int iterable
        shape of the input, excluding batch size
    L : int list or int
        split coefficients for each mode, excluding batch size
    
    References
    ----------
     N/A
    """

  def __init__(self, input_size, L, batch_size, map_type = 1, device=None, dtype=None):
      # Input contains the batch size mode
      # L doesnt contain the batch size mode

      super().__init__()

      if isinstance(input_size, int):
        self.input_size = (input_size, )
      else:
        self.input_size = tuple(input_size)
      if isinstance(L, int):
        self.L = (L, )
      else:
        self.L = tuple(L)
      if map_type == 1 or self.L.count(2) != len(self.L):
          self.map_type = 1
      else:
          self.map_type = 2

      self.device = device
      self.dtype  = dtype
      self.batch_size = batch_size


      # many operations in init

      self.M = []
      for i,_ in enumerate(self.L):
          if self.input_size[i] % self.L[i] != 0:
              raise RuntimeError(f'Bad L : input_size {self.input_size[i]} is not divisible by L size {self.L[i]}')
          self.M.append(int(self.input_size[i] / self.L[i]))

      self.lists_indices = []
      for i in range(len((list(self.L) + self.M))):
          self.lists_indices.append(np.arange((list(self.L) + self.M)[i]))


      self.indices = np.array([p for p in itertools.product(*self.lists_indices)])

      self.new_shapes = tuple([self.batch_size] + list(self.L) + self.M)

      self.out = torch.empty(self.new_shapes, device=self.device, dtype=self.dtype)
      

  def forward(self, x):
    """Performs a forward pass"""
    
    # generate output shape
    
    # Already generated in init

    # generate all possible combination of indices

    # Already generated in init

    # Mapping

    if self.map_type == 1:
      
      for index_B in self.indices:

          # Input is : x
          # Output is : out
          # Maps via exec() command
          ### Command to execute
          code = '' + 'self.out[:,'
          map_B_string = ','.join(map(str, index_B))
          code += map_B_string + ']'
          ### Command to execute

          index_A = []
          for ii in range(len(index_B)):
              index_A.append(index_B[ii]*self.M[ii] +  index_B[ii+len(self.L)])
              if ii == len(self.L) - 1:
                  ### Command to execute
                  map_A_string = ','.join(map(str, index_A))
                  code += '= x[:,'
                  code += map_A_string + ']'
                  ### Command to execute
                  # If the original dim is 3D then the 
                  # code is similar to : out[:,0,0,0,0,0,0] = x[:,0,0,0]
                  exec(code)
                  break  

      return self.out
      
    else:

      for index_B in self.indices:

          # Input is : x
          # Output is : out
          # Maps via exec() command
          ### Command to execute
          code = '' + 'self.out[:,'
          map_B_string = ','.join(map(str, index_B))
          code += map_B_string + ']'
          ### Command to execute

          index_A = []
          for ii in range(len(index_B)):
              index_A.append(index_B[ii + len(self.L)]*2 +  index_B[ii])
              if ii == len(self.L) - 1:
                  ### Command to execute
                  map_A_string = ','.join(map(str, index_A))
                  code += '= x[:,'
                  code += map_A_string + ']'
                  ### Command to execute
                  # If the original dim is 3D then the 
                  # code is similar to : out[:,0,0,0,0,0,0] = x[:,0,0,0]
                  exec(code)
                  break  
      return self.out


# Version 3 of chunk reshape

In [130]:
class chunkReshape_version3(nn.Module):
  """Tensor higher dimensional reshape- smaller chunk reshape

    Parameters
    ----------
    input_size : int iterable
        shape of the input, excluding batch size
    L : int list or int
        split coefficients for each mode, excluding batch size
    
    References
    ----------
     N/A
    """

  def __init__(self, input_size, L, batch_size, map_type = 1, device=None, dtype=None):
      # Input contains the batch size mode
      # L doesnt contain the batch size mode

      super().__init__()

      if isinstance(input_size, int):
        self.input_size = (input_size, )
      else:
        self.input_size = tuple(input_size)
      if isinstance(L, int):
        self.L = (L, )
      else:
        self.L = tuple(L)
      if map_type == 1 or self.L.count(2) != len(self.L):
          self.map_type = 1
      else:
          self.map_type = 2

      self.device = device
      self.dtype  = dtype
      self.batch_size = batch_size


      # many operations in init

      self.M = []
      for i,_ in enumerate(self.L):
          if self.input_size[i] % self.L[i] != 0:
              raise RuntimeError(f'Bad L : input_size {self.input_size[i]} is not divisible by L size {self.L[i]}')
          self.M.append(int(self.input_size[i] / self.L[i]))

      self.lists_indices = []
      for i in range(len((list(self.L) + self.M))):
          self.lists_indices.append(np.arange((list(self.L) + self.M)[i]))


      self.indices = np.array([p for p in itertools.product(*self.lists_indices)])

      self.new_shapes = tuple([self.batch_size] + list(self.L) + self.M)

      #self.out = torch.empty(self.new_shapes, device=self.device, dtype=self.dtype)
      

  def forward(self, x):
    """Performs a forward pass"""
    
    # generate output shape
    
    out = x.reshape(self.new_shapes)

    # generate all possible combination of indices

    # Already generated in init

    # Mapping

    if self.map_type == 1:
      
      for index_B in self.indices:

          # Input is : x
          # Output is : out
          # Maps via exec() command
          ### Command to execute
          code = '' + 'out[:,'
          map_B_string = ','.join(map(str, index_B))
          code += map_B_string + ']'
          ### Command to execute

          index_A = []
          for ii in range(len(index_B)):
              index_A.append(index_B[ii]*self.M[ii] +  index_B[ii+len(self.L)])
              if ii == len(self.L) - 1:
                  ### Command to execute
                  map_A_string = ','.join(map(str, index_A))
                  code += '= x[:,'
                  code += map_A_string + ']'
                  ### Command to execute
                  # If the original dim is 3D then the 
                  # code is similar to : out[:,0,0,0,0,0,0] = x[:,0,0,0]
                  exec(code)
                  break  

      return out
      
    else:

      for index_B in self.indices:

          # Input is : x
          # Output is : out
          # Maps via exec() command
          ### Command to execute
          code = '' + 'out[:,'
          map_B_string = ','.join(map(str, index_B))
          code += map_B_string + ']'
          ### Command to execute

          index_A = []
          for ii in range(len(index_B)):
              index_A.append(index_B[ii + len(self.L)]*2 +  index_B[ii])
              if ii == len(self.L) - 1:
                  ### Command to execute
                  map_A_string = ','.join(map(str, index_A))
                  code += '= x[:,'
                  code += map_A_string + ']'
                  ### Command to execute
                  # If the original dim is 3D then the 
                  # code is similar to : out[:,0,0,0,0,0,0] = x[:,0,0,0]
                  exec(code)
                  break  
      return out


# MNIST Dataset

- A Large batch size is required to notice the difference between models in <code> forward() </code>

- Defined as function to generalize in <b> Report Section </b>


In [5]:
# Big Batch_size to notice the difference

# Defined as a function to generalize in Report Section

# to run on CPU, uncomment the following line:
#device = 'cpu'

MNIST_train_dataset = datasets.MNIST('./data/', train=True, download=True,
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        ])) 
MNIST_test_dataset = datasets.MNIST('./data/', train=False, download=True,
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        ]))


def MNIST_loader(_batch_size = 200 ,_device = 'cuda'):

    batch_size = _batch_size
    device = _device   

    train_loader_mnist = torch.utils.data.DataLoader(MNIST_train_dataset,
          batch_size=batch_size, shuffle=True)
    test_loader_mnist = torch.utils.data.DataLoader(MNIST_test_dataset,
          batch_size=batch_size, shuffle=True)
    
    return batch_size, device, train_loader_mnist, test_loader_mnist


# Initialize 
MNIST_loader()

100%|██████████| 9912422/9912422 [00:00<00:00, 203423579.82it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 27637621.22it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 68121424.67it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 15388149.25it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



(200,
 'cuda',
 <torch.utils.data.dataloader.DataLoader at 0x7f1c1c7eb640>)

## Example of Chunk Reshape of a Tensor - MNIST

In [6]:
# Dataset Loaders
batch_size, device, train_loader_mnist, test_loader_mnist = MNIST_loader()


# Get a batch from dataset
examples_mnist = enumerate(train_loader_mnist)
batch_idx, (example_data_mnist, example_targets_mnist) = next(examples_mnist)


# One forward through Net
print('Input shape ', example_data_mnist.shape)
conv1 = nn.Conv2d(1, 20, kernel_size=5)
x1 = F.relu(F.max_pool2d(conv1(example_data_mnist), 2))
print('Input shape after convolution kernel 5 and max pooling kernel 2 ', x1.shape)
conv2 = nn.Conv2d(20, 50, kernel_size=5)
x2 = F.relu(F.max_pool2d(conv2(x1), 2))
print('Input shape after another convolution kernel 5 and max pooling kernel 2 ', x2.shape)
print('Ready for Reshape Using L = [2,2,2] and map type 1')
# With Reshape
Chunk_Reshape = chunkReshape(input_size = [50,4,4], L = [2,2,2], map_type = 1)
x3 = Chunk_Reshape(x2)
print('output size of Chunk Reshape', x3.shape)
# TCL 
tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
x4 = F.relu(tcl(x3))
print('output size of TCL', x4.shape)
x5 = x4.squeeze()
print('output size of TCL with squeeze() ', x5.shape)
# TRL 
trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])
x6 = trl(x5)
print('output size of TRL', x6.shape)

Input shape  torch.Size([200, 1, 28, 28])
Input shape after convolution kernel 5 and max pooling kernel 2  torch.Size([200, 20, 12, 12])
Input shape after another convolution kernel 5 and max pooling kernel 2  torch.Size([200, 50, 4, 4])
Ready for Reshape Using L = [2,2,2] and map type 1
output size of Chunk Reshape torch.Size([200, 2, 2, 2, 25, 2, 2])
output size of TCL torch.Size([200, 1, 1, 1, 13, 2, 2])
output size of TCL with squeeze()  torch.Size([200, 13, 2, 2])
output size of TRL torch.Size([200, 10])


# CIFAR10 Dataset

- Same Batch size same Issues as MNIST

In [7]:
# Big Batch_size to notice the difference

# Defined as a function to generalize in Report Section

# to run on CPU, uncomment the following line:
#device = 'cpu'

CIFAR10_train_dataset = datasets.CIFAR10('./data/', train=True, download=True,
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                        ])) 
CIFAR10_test_dataset = datasets.CIFAR10('./data/', train=False, download=True,
                        transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                        ]))



def CIFAR10_loader(_batch_size = 200 ,_device = 'cuda'):

    batch_size = _batch_size
    device = _device   

    train_loader_cifar10 = torch.utils.data.DataLoader(CIFAR10_train_dataset,
          batch_size=batch_size, shuffle=True)
    test_loader_cifar10 = torch.utils.data.DataLoader(CIFAR10_test_dataset,
          batch_size=batch_size, shuffle=True)
    
    return batch_size, device, train_loader_cifar10, test_loader_cifar10


# Initialize 
CIFAR10_classes = ['plane', 'car', 'bird', 'cat',
                  'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
CIFAR10_loader()

100%|██████████| 170498071/170498071 [00:05<00:00, 29597536.96it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data/
Files already downloaded and verified


(200,
 'cuda',
 <torch.utils.data.dataloader.DataLoader at 0x7f1c1c85a9e0>)

## Example of Chunk Reshape of a Tensor - CIFAR10

In [8]:
# Dataset Loaders
batch_size, device, train_loader_cifar10, test_loader_cifar10 = CIFAR10_loader()

# Get a batch from dataset
examples_cifar10 = enumerate(train_loader_cifar10)
batch_idx, (example_data_cifar10, example_targets_cifar10) = next(examples_cifar10)

print('Input shape ', example_data_cifar10.shape)
conv1 = nn.Conv2d(3, 20, kernel_size=5)
x1 = F.relu(F.max_pool2d(conv1(example_data_cifar10), 2))
print('Input shape after convolution kernel 5 and max pooling kernel 2 ', x1.shape)
conv2 = nn.Conv2d(20, 50, kernel_size=6)
x2 = F.relu(F.max_pool2d(conv2(x1), 2))
print('Input shape after another convolution kernel 6 and max pooling kernel 2 ', x2.shape)
print('Ready for Reshape Using L = [2,2,2] and map type 1')
# With Reshape
Chunk_Reshape = chunkReshape(input_size = [50,4,4], L = [2,2,2], map_type = 1)
x3 = Chunk_Reshape(x2)
print('output size of Chunk Reshape', x3.shape)
# TCL 
tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
x4 = F.relu(tcl(x3))
print('output size of TCL', x4.shape)
x5 = x4.squeeze()
print('output size of TCL with squeeze() ', x5.shape)
# TRL 
trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])
x6 = trl(x5)
print('output size of TRL', x6.shape)

Input shape  torch.Size([200, 3, 32, 32])
Input shape after convolution kernel 5 and max pooling kernel 2  torch.Size([200, 20, 14, 14])
Input shape after another convolution kernel 6 and max pooling kernel 2  torch.Size([200, 50, 4, 4])
Ready for Reshape Using L = [2,2,2] and map type 1
output size of Chunk Reshape torch.Size([200, 2, 2, 2, 25, 2, 2])
output size of TCL torch.Size([200, 1, 1, 1, 13, 2, 2])
output size of TCL with squeeze()  torch.Size([200, 13, 2, 2])
output size of TRL torch.Size([200, 10])


# Create Networks

## MNIST Models

- Base Model
- TRL only Model
- TCL + TRL Model
- Chunk Reshape map type 1 + TCL + TRL Model
- Chunk Reshape map type 2 + TCL + TRL Model

In [15]:

# Base Model

class MNIST_Net_base(nn.Module):
    def __init__(self):
        super(MNIST_Net_base, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(800, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1,800)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, 1)


# TRL only Model

class MNIST_Net_TRL(nn.Module):
    def __init__(self):
        super(MNIST_Net_TRL, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.trl = tltorch.TRL(input_shape = [50,4,4] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.trl(x)
        return F.log_softmax(x, 1)


# TCL + TRL Model

class MNIST_Net_TCL_TRL(nn.Module):
    def __init__(self):
        super(MNIST_Net_TCL_TRL, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.tcl = tltorch.TCL(input_shape= [50,4,4], rank=[30,2,2])
        self.trl = tltorch.TRL(input_shape = [30,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(self.tcl(x))
        x = self.trl(x)
        return F.log_softmax(x, 1)

# Chunk Reshape map type 1 + TCL + TRL Model

class MNIST_Net_chunk_TCL_TRL_map1(nn.Module):
    def __init__(self):
        super(MNIST_Net_chunk_TCL_TRL_map1, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.chunk = chunkReshape(input_size = [50,4,4], L = [2,2,2], map_type = 1, device = 'cuda')
        # to make it easier we contract L values to rank 1
        self.tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
        self.trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.chunk(x)
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)



# Chunk Reshape map type 2 + TCL + TRL Model

class MNIST_Net_chunk_TCL_TRL_map2(nn.Module):
    def __init__(self):
        super(MNIST_Net_chunk_TCL_TRL_map2, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.chunk = chunkReshape(input_size = [50,4,4], L = [2,2,2], map_type = 2, device = 'cuda')
        # to make it easier we contract L values to rank 1
        self.tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
        self.trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.chunk(x)
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)


# Chunk Reshape map type 1 + TCL + TRL Model version 2 of chunk reshape

class MNIST_Net_chunk_TCL_TRL_map1_version2(nn.Module):
    def __init__(self, _batch_size):
        super(MNIST_Net_chunk_TCL_TRL_map1_version2, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.chunk = chunkReshape_version2(input_size = [50,4,4], L = [2,2,2], batch_size= _batch_size , map_type = 1, device = 'cuda')
        # to make it easier we contract L values to rank 1
        self.tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
        self.trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.chunk(x)
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)



# Chunk Reshape map type 2 + TCL + TRL Model version 2 of chunk reshape

class MNIST_Net_chunk_TCL_TRL_map2_version2(nn.Module):
    def __init__(self, _batch_size):
        super(MNIST_Net_chunk_TCL_TRL_map2_version2, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=5)
        self.chunk = chunkReshape_version2(input_size = [50,4,4], L = [2,2,2], batch_size = _batch_size, map_type = 2, device = 'cuda')
        # to make it easier we contract L values to rank 1
        self.tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
        self.trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.chunk(x)
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)


## CIFAR10 Models

- Base Model
- TRL only Model
- TCL + TRL Model
- Chunk Reshape map type 1 + TCL + TRL Model
- Chunk Reshape map type 2 + TCL + TRL Model

In [16]:

# Base Model

class CIFAR10_Net_base(nn.Module):
    def __init__(self):
        super(CIFAR10_Net_base, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=6)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(800, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1,800)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, 1)


# TRL only Model

class CIFAR10_Net_TRL(nn.Module):
    def __init__(self):
        super(CIFAR10_Net_TRL, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=6)
        self.trl = tltorch.TRL(input_shape = [50,4,4] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.trl(x)
        return F.log_softmax(x, 1)


# TCL + TRL Model

class CIFAR10_Net_TCL_TRL(nn.Module):
    def __init__(self):
        super(CIFAR10_Net_TCL_TRL, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=6)
        self.tcl = tltorch.TCL(input_shape= [50,4,4], rank=[30,2,2])
        self.trl = tltorch.TRL(input_shape = [30,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(self.tcl(x))
        x = self.trl(x)
        return F.log_softmax(x, 1)

# Chunk Reshape map type 1 + TCL + TRL Model

class CIFAR10_Net_chunk_TCL_TRL_map1(nn.Module):
    def __init__(self):
        super(CIFAR10_Net_chunk_TCL_TRL_map1, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=6)
        self.chunk = chunkReshape(input_size = [50,4,4], L = [2,2,2], map_type = 1, device = 'cuda')
        # to make it easier we contract L values to rank 1
        self.tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
        self.trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.chunk(x)
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)



# Chunk Reshape map type 2 + TCL + TRL Model

class CIFAR10_Net_chunk_TCL_TRL_map2(nn.Module):
    def __init__(self):
        super(CIFAR10_Net_chunk_TCL_TRL_map2, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=6)
        self.chunk = chunkReshape(input_size = [50,4,4], L = [2,2,2], map_type = 2, device = 'cuda')
        # to make it easier we contract L values to rank 1
        self.tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
        self.trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.chunk(x)
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)


# Chunk Reshape map type 1 + TCL + TRL Model version 2 of chunk reshape

class CIFAR10_Net_chunk_TCL_TRL_map1_version2(nn.Module):
    def __init__(self, _batch_size):
        super(CIFAR10_Net_chunk_TCL_TRL_map1_version2, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=6)
        self.chunk = chunkReshape_version2(input_size = [50,4,4], L = [2,2,2], batch_size = _batch_size, map_type = 1, device = 'cuda')
        # to make it easier we contract L values to rank 1
        self.tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
        self.trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.chunk(x)
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)



# Chunk Reshape map type 2 + TCL + TRL Model version 2 of chunk reshape

class CIFAR10_Net_chunk_TCL_TRL_map2_version2(nn.Module):
    def __init__(self, _batch_size):
        super(CIFAR10_Net_chunk_TCL_TRL_map2_version2, self).__init__()
        self.conv1 = nn.Conv2d(3, 20, kernel_size=5)
        self.conv2 = nn.Conv2d(20, 50, kernel_size=6)
        self.chunk = chunkReshape_version2(input_size = [50,4,4], L = [2,2,2], batch_size = _batch_size, map_type = 2, device = 'cuda')
        # to make it easier we contract L values to rank 1
        self.tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
        self.trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = self.chunk(x)
        x = F.relu(self.tcl(x)).squeeze()
        x = self.trl(x)
        return F.log_softmax(x, 1)

# Models, Optimizers ,and Criterions

In [17]:
# MNIST 


MNIST_Models = [
    MNIST_Net_base(),
    MNIST_Net_TRL(),
    MNIST_Net_TCL_TRL(),
    MNIST_Net_chunk_TCL_TRL_map1(),
    MNIST_Net_chunk_TCL_TRL_map2(), 
]


MNIST_Optimizers = [
    optim.SGD(MNIST_Models[0].parameters(), lr=0.01, momentum=0.9),
    optim.SGD(MNIST_Models[1].parameters(), lr=0.01, momentum=0.9),
    optim.SGD(MNIST_Models[2].parameters(), lr=0.01, momentum=0.9),
    optim.SGD(MNIST_Models[3].parameters(), lr=0.01, momentum=0.9),
    optim.SGD(MNIST_Models[4].parameters(), lr=0.01, momentum=0.9)
]

MNIST_Criterion = [
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss()
]


# CIFAR10

CIFAR10_Models= [
    CIFAR10_Net_base(),
    CIFAR10_Net_TRL(),
    CIFAR10_Net_TCL_TRL(),
    CIFAR10_Net_chunk_TCL_TRL_map1(),
    CIFAR10_Net_chunk_TCL_TRL_map2()
]

CIFAR10_Optimizers = [
    optim.SGD(CIFAR10_Models[0].parameters(), lr=0.01, momentum=0.9),
    optim.SGD(CIFAR10_Models[1].parameters(), lr=0.01, momentum=0.9),
    optim.SGD(CIFAR10_Models[2].parameters(), lr=0.01, momentum=0.9),
    optim.SGD(CIFAR10_Models[3].parameters(), lr=0.01, momentum=0.9),
    optim.SGD(CIFAR10_Models[4].parameters(), lr=0.01, momentum=0.9)
]

CIFAR10_Criterion = [
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss(),
    nn.CrossEntropyLoss()
]



# Check The Results for one Forward pass on MNIST and CIFAR10

In [18]:
models_names = ['base', 'TRL only', 'TCL + TRL', 'Chunk Reshape map 1 + TCL + TRL', 'Chunk Reshape map 2 + TCL + TRL']

## MNIST (Report on a single batch forward operation)

In [19]:
# MNIST

for index,_model in enumerate(MNIST_Models):
    model = _model.to(device)
    st = time.time()
    output = model(example_data_mnist.to(device)).to(device)
    elapsed_time = time.time() - st
    #time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    print(f'#### Execution time one forward pass on batch size ({batch_size}) MNIST Dataset is {elapsed_time} Model Type : {models_names[index]}')


#### Execution time one forward pass on batch size (200) MNIST Dataset is 0.9950940608978271 Model Type : base
#### Execution time one forward pass on batch size (200) MNIST Dataset is 0.03965401649475098 Model Type : TRL only
#### Execution time one forward pass on batch size (200) MNIST Dataset is 0.0020074844360351562 Model Type : TCL + TRL
#### Execution time one forward pass on batch size (200) MNIST Dataset is 0.13838911056518555 Model Type : Chunk Reshape map 1 + TCL + TRL
#### Execution time one forward pass on batch size (200) MNIST Dataset is 0.0968160629272461 Model Type : Chunk Reshape map 2 + TCL + TRL


## CIFAR10  (Report on a single batch forward operation)

In [20]:
# CIFAR10

for index,_model in enumerate(CIFAR10_Models):
    model = _model.to(device)
    st = time.time()
    output = model(example_data_cifar10.to(device)).to(device)
    elapsed_time = time.time() - st
    # time.strftime("%H:%M:%S", time.gmtime(elapsed_time))
    print(f'#### Execution time one forward pass on batch size ({batch_size}) CIFAR10 Dataset is {elapsed_time} Model Type : {models_names[index]}')





#### Execution time one forward pass on batch size (200) CIFAR10 Dataset is 0.017882108688354492 Model Type : base
#### Execution time one forward pass on batch size (200) CIFAR10 Dataset is 0.0016760826110839844 Model Type : TRL only
#### Execution time one forward pass on batch size (200) CIFAR10 Dataset is 0.001870870590209961 Model Type : TCL + TRL
#### Execution time one forward pass on batch size (200) CIFAR10 Dataset is 0.08877682685852051 Model Type : Chunk Reshape map 1 + TCL + TRL
#### Execution time one forward pass on batch size (200) CIFAR10 Dataset is 0.08987975120544434 Model Type : Chunk Reshape map 2 + TCL + TRL


# Report Section

In this section we report different forward passes on different batch sizes and different instances in a csv sheet.

In [ ]:
# PATH
DIRECTORY = './Report'
PATH = DIRECTORY + '/reportV3.5.0.csv'
if not os.path.exists(DIRECTORY):
   os.mkdir(DIRECTORY)

# Dataframe
columns = [
    'Dataset',
    'batch_size',
    'model',
    'forward pass time'
]

dataframe = pd.DataFrame(columns = columns )

datasets = ['MNIST', 'CIFAR10']
batch_sizes = [16, 200, 1000, 20000, 60000]
# models_names = ['base', 'TRL only', 'TCL + TRL', 'Chunk Reshape map 1 + TCL + TRL', 'Chunk Reshape map 2 + TCL + TRL']

for dataset in datasets:
    # Get the dataset
    for i, model_name in enumerate(models_names):
        # Get model and its name
        for b in batch_sizes:
            # Set a batch size
            if dataset == 'MNIST':
                # Dataset loaders and models
                batch_size, device, train_loader, test_loader = MNIST_loader(_batch_size = b, _device = 'cuda')
                model = MNIST_Models[i].to(device)
            else:
                # Dataset loaders and models
                batch_size, device, train_loader, test_loader = CIFAR10_loader(_batch_size = b, _device = 'cuda')
                model = CIFAR10_Models[i].to(device)
            
            # Get one batch of dataset
            examples = enumerate(train_loader)
            batch_idx, (example_data, example_targets) = next(examples)

            # Calculate time spent on forward pass
            st = time.time()
            output = model(example_data.to(device)).to(device)
            elapsed_time = time.time() - st


            # dataframe entry
            entry = {
                columns[0]: [dataset],
                columns[1]: [batch_size],
                columns[2]: [model_name],
                columns[3]: [elapsed_time]
            }
            # append to dataframe
            dataframe = pd.concat([dataframe, pd.DataFrame(entry)], ignore_index = True )
            dataframe.reset_index()

    #break # To only display MNIST results 

# Save to file
dataframe.to_csv(PATH)
# show dataframe
display(dataframe)

# Study The case of inconsistency in forward time

- Possible Cause : Not Optimal implementation of TCL or/and TRL in tensorly-torch package (status : <font color ='red' > Rejected</font> ) 

- Possible Cause :  Not Optimal Implementation or Reshape forward function (status : <font color = 'yellow'> Pending</font>)
  - Itertools
  - exec()

- Built-in reshape function is much faster and performs as expected but the map it provides is wrong. Overall it is faster than original 3d TCL but with a wrong mapping.

- Version 2 of chunk reshape (status : <font color = 'red'> Rejected </font>) 
  - It is faster but not as fast as built-in reshape function and overall still takes longer to run than original 3d TCL.

- Version 3 of chunk reshape (status : <font color = 'red'> Rejected </font>)
  - It is still faster than version 2 or the original as it generate the output from the reshaped version of input but it still takes longer to run than built-in reshape function

In [132]:
tempstring = "\n######################################\n\n"

# Dataset Loaders
batch_size, device, train_loader_mnist, test_loader_mnist = MNIST_loader(_batch_size = 15000, _device = 'cuda')


# Get a batch from dataset
examples_mnist = enumerate(train_loader_mnist)
batch_idx, (example_data_mnist, example_targets_mnist) = next(examples_mnist)


# One forward through Net
conv1 = nn.Conv2d(1, 20, kernel_size=5)
st = time.time()
x1 = F.relu(F.max_pool2d(conv1(example_data_mnist), 2))
elapsed_time = time.time() - st
print('Time spent on first convolution Layer ', elapsed_time)


conv2 = nn.Conv2d(20, 50, kernel_size=5)
st = time.time()
x2 = F.relu(F.max_pool2d(conv2(x1), 2))
elapsed_time = time.time() - st
print('Time spent on second convolution Layer ', elapsed_time)

print(tempstring)
# For further studies (and for further simplification for viewers)
x2_3d = x2
x2_reshape = x2
x2_version2 = x2
x2_version3 = x2


# With Reshape
Chunk_Reshape = chunkReshape(input_size = [50,4,4], L = [2,2,2], map_type = 1)
st = time.time()
x3 = Chunk_Reshape(x2)
elapsed_time = time.time() - st
print('Time spent on Reshaping the Tensor ', elapsed_time)


# TCL 
tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
st_1 = time.time()
x4 = F.relu(tcl(x3))
elapsed_time_1 = time.time() - st_1
print('Time spent on TCL 6 Dimensional ', elapsed_time_1)

st_2 = time.time()
x5 = x4.squeeze()
elapsed_time_2 = time.time() - st_2
print('Time spent on squeeze() ', elapsed_time_2)

elapsed_time = time.time() - st_1
print('Total Time spent TCL 6 dimensional and squeeze() ', elapsed_time)


# Total time spent on Reshape + TCL + squeeze()
elapsed_time = time.time() - st
print('Total Time spent on Reshape + TCL 6 dimensional +  squeeze() ', elapsed_time)

print(tempstring)
# Case of TCL with 3 dimension

tcl_3d = tltorch.TCL(input_shape= [50,4,4], rank=[30,2,2])
st = time.time()
x4_3d = F.relu(tcl_3d(x2_3d))
elapsed_time = time.time() - st
print('Time spent on TCL 3 Dimensional ', elapsed_time)

print(tempstring)
# In case of using reshape in-built function of Tensorly or torch
# The result is Not correct but we wish to calculate the time spent
st = time.time()
x3_reshape = x2_reshape.reshape((batch_size, 2, 2, 2, 25, 2, 2))
elapsed_time_1 = time.time() - st
print('Reshape time using Tensoly Reshape function : ', elapsed_time_1)
st_1 = time.time()
x4_reshape = F.relu(tcl(x3_reshape))
elapsed_time_2 = time.time() - st_1
print('Time spent on TCL 6 Dimensional ', elapsed_time_2)
st_2 = time.time()
x5_reshape = x4_reshape.squeeze()
elapsed_time_3 = time.time() - st_2
print('Total Time spent squeeze() ', elapsed_time_3)
elapsed_time = time.time() - st
print('Total Time spent on Reshape + TCL 6 dimensional +  squeeze() with .reshape function ', elapsed_time)

print(tempstring)
# In case we are using version 2 of chunk reshape
# With Reshape version2
Chunk_Reshape = chunkReshape_version2(input_size = [50,4,4], L = [2,2,2], batch_size = batch_size, map_type = 1)
st = time.time()
x3_version2 = Chunk_Reshape(x2_version2)
elapsed_time = time.time() - st
print('Time spent on Reshaping the Tensor (version2) ', elapsed_time)


# TCL 
tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
st_1 = time.time()
x4_version2 = F.relu(tcl(x3_version2))
elapsed_time_1 = time.time() - st_1
print('Time spent on TCL 6 Dimensional (version2) ', elapsed_time_1)

st_2 = time.time()
x5_version2 = x4_version2.squeeze()
elapsed_time_2 = time.time() - st_2
print('Time spent on squeeze() (version2) ', elapsed_time_2)

elapsed_time = time.time() - st_1
print('Total Time spent TCL 6 dimensional and squeeze() (version2) ', elapsed_time)


# Total time spent on Reshape + TCL + squeeze()
elapsed_time = time.time() - st
print('Total Time spent on Reshape + TCL 6 dimensional +  squeeze() (version 2) ', elapsed_time)

print(tempstring)
# In case we are using version 3 of chunk reshape
# With Reshape version2
Chunk_Reshape = chunkReshape_version3(input_size = [50,4,4], L = [2,2,2], batch_size = batch_size, map_type = 1)
st = time.time()
x3_version3 = Chunk_Reshape(x2_version3)
elapsed_time = time.time() - st
print('Time spent on Reshaping the Tensor (version3) ', elapsed_time)


# TCL 
tcl = tltorch.TCL(input_shape= [2,2,2,25,2,2], rank=[1,1,1,13,2,2])
st_1 = time.time()
x4_version3 = F.relu(tcl(x3_version3))
elapsed_time_1 = time.time() - st_1
print('Time spent on TCL 6 Dimensional (version3) ', elapsed_time_1)

st_2 = time.time()
x5_version3 = x4_version3.squeeze()
elapsed_time_2 = time.time() - st_2
print('Time spent on squeeze() (version3) ', elapsed_time_2)

elapsed_time = time.time() - st_1
print('Total Time spent TCL 6 dimensional and squeeze() (version3) ', elapsed_time)


# Total time spent on Reshape + TCL + squeeze()
elapsed_time = time.time() - st
print('Total Time spent on Reshape + TCL 6 dimensional +  squeeze() (version 3) ', elapsed_time)

print(tempstring)
# TRL
# TRL input in both cases (with/without reshape) is 3 dimensional 
# But the ranks are different
trl = tltorch.TRL(input_shape = [13,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])
st = time.time()
x6 = trl(x5)
elapsed_time = time.time() - st
print('Time spent on TRL (input rank : [13,2,2]) ', elapsed_time)


trl_3d = tltorch.TRL(input_shape = [30,2,2] , output_shape = [10], factorization = 'Tucker', rank = [10,3,3,10])
st = time.time()
x6_3d = trl_3d(x4_3d)
elapsed_time = time.time() - st
print('Time spent on TRL (input rank : [30,2,2]) ', elapsed_time)


Time spent on first convolution Layer  3.1734468936920166
Time spent on second convolution Layer  1.7070107460021973

######################################


Time spent on Reshaping the Tensor  0.35261011123657227
Time spent on TCL 6 Dimensional  0.06156492233276367
Time spent on squeeze()  0.0011353492736816406
Total Time spent TCL 6 dimensional and squeeze()  0.0630791187286377
Total Time spent on Reshape + TCL 6 dimensional +  squeeze()  0.4191412925720215

######################################


Time spent on TCL 3 Dimensional  0.09839797019958496

######################################


Reshape time using Tensoly Reshape function :  0.001592397689819336
Time spent on TCL 6 Dimensional  0.07184123992919922
Total Time spent squeeze()  0.0012443065643310547
Total Time spent on Reshape + TCL 6 dimensional +  squeeze() with .reshape function  0.07605361938476562

######################################


Time spent on Reshaping the Tensor (version2)  0.3576040267944336
Time spent on 

 It is observed that .reshape function performs faster, therefore it is advised to study the base code of .reshape to further improve our Reshape class and forward pass time. 


 Version 2 still takes longer to execute than original 3D TCL, therefore it has been rejected.  

In [91]:
# torch.as_stride
# torch.index_select


# tensor = torch.Tensor(np.arange(0,100).reshape(10,10))
# temp1 = torch.index_select(tensor,1, torch.tensor(range(0,5)))
# torch.index_select(temp1, 0, torch.tensor(range(0,5)))

In [126]:
#import tensorflow as tf





SyntaxError: ignored

In [ ]:
tensor = torch.Tensor(np.arange(0,100).reshape(10,10))
tensor.reshape(2,2,5,5)